In [189]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import json

In [190]:
with open("source_texts/bible/luther_old_testament.xml", "r") as f:
    bible_ot = f.read()

In [191]:
soup = BeautifulSoup(bible_ot, 'xml')

In [192]:
def chunk_verses(verse_text: str) -> list:
    all_chunks = []
    chunks = re.split('/|,|\.|:', verse_text)
    word_collector = ""
    for chunk in chunks:
        word_collector += " " + chunk.strip()
        if len(word_collector.split()) >= 4:
            all_chunks.append(word_collector.strip())
            word_collector = ""
    return all_chunks

<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_518/2323431692.py:3: SyntaxWarning: invalid escape sequence '\.'
  chunks = re.split('/|,|\.|:', verse_text)


In [193]:
old_testament = {}
books = soup.find_all("BIBLEBOOK")

for book in books:
    booktitle = book['bname']
    filename = f"source_texts/bible/old_testament/{re.sub(" ", "_", booktitle)}.json"
    chapters = {}
    for chap_nr in range(len(book.find_all("CHAPTER"))):
        chapter = book.find_all("CHAPTER")[chap_nr]
        verses = {}
        for verse_nr in range(len(chapter.find_all("VERS"))):
            verse = chapter.find_all("VERS")[verse_nr]
            for note in verse.find_all("NOTE"):                     # remove annotations
                note.string.replace_with(" ")
            verstext = verse.get_text()
            verse_chunked = chunk_verses(verstext)
            verses[verse_nr + 1] = verse_chunked
        chapters[chap_nr + 1] = verses
    old_testament[booktitle] = chapters

with open("source_texts/bible/old_testament_chunked.json", "w") as f:
    json.dump(old_testament, f,ensure_ascii=False)


In [181]:
with open("source_texts/bible/luther_septembertestament_1522.TEI-P5.xml", "r") as f:
    bible_tei = f.read()

In [182]:
rep = {"&#x017F;": "s",
       "&#xA75B;": "r",
       "vn&#x0303;": "vnd",
       "&#x0303;": "n",
       "<lb/>": " "
            }

rep = dict((re.escape(k), v) for k, v in rep.items()) 
pattern = re.compile("|".join(rep.keys()))
bible_tei = pattern.sub(lambda m: rep[re.escape(m.group(0))], bible_tei)

bible_tei = re.sub(r'<ref target="https:.*<\/ref>', " ◊ ", bible_tei)

soup = BeautifulSoup(bible_tei, 'xml')

In [183]:
bible_nt_books = ["Mt", "Mk", "Lk", "Joh",
                  "Apg", "Röm", "Kor1", "Kor2",
                  "Gal", "Eph", "Phil", "Kol",
                  "Thess1", "Thess2", "Tim1", "Tim2",
                  "Tit", "Phlm", "Petr1", "Petr2",
                  "Joh1", "Joh2", "Joh3", "Hebr",
                  "Jak", "Jud", "Offb"]

In [184]:
for note in soup.find_all("note"):      # remove all comments not editorial
    if note.get("type") != "editorial":
        note.decompose()

for fw in soup.find_all("fw"):
    fw.decompose()

for head in soup.find_all("head"):
    head.decompose()

for sic in soup.find_all("sic"):
    sic.decompose()

for reg in soup.find_all("reg"):
    reg.decompose()

In [185]:
books = soup.find_all("div", {"xml:id": lambda x: x in bible_nt_books})

In [ ]:
new_testament = {}

url = re.compile(r'https?://\S+')

for book in books:
    booktitle = book["xml:id"]
    filename = f"source_texts/bible/new_testament/{booktitle}.json"
    chapters = {}
    for chap_nr in range(len(book.find_all("div"))):
        chapter = book.find_all("div")[chap_nr]
        text = chapter.get_text(strip=True)
        text = re.sub("\n", "", text)
        text = re.sub(r"- ", "", text)
        text = re.sub("-", "", text)

        verse_nr = 0
        verses = {}
        
        split_text = text.split("◊")
        for elem in split_text:
            if elem != "":
                verse_nr += 1
                verse_chunked = chunk_verses(elem)
                verses[verse_nr] = verse_chunked
        chapters[chap_nr + 1] = verses
        vers_nr = 0
    new_testament[booktitle] = chapters



In [188]:
with open("source_texts/bible/new_testament_chunked.json", "w") as f:
    json.dump(new_testament, f,ensure_ascii=False)